In [43]:
# These are the standard torch imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import random
import math
import time
# Helper libraries
import random
from tqdm import tqdm
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data

In [53]:
data = pd.read_csv('D:/IITM/DeepCache/syntheticDataset_o50.csv')
data.head()

,object_ID,request_time
0,5,2.961472
1,25,3.274127
2,2,3.785475
3,2,4.455687
4,4,5.288994


In [77]:
data = np.array(data)
req = data[:,0]
time = data[:,1]

In [78]:
len(req)

292146

In [124]:
batch_size = 20000 #10% of training data

In [127]:
train_req = req[:200000].reshape(-1,batch_size) # total batches = 10

target_req = []
for i in range(1,11):
    temp = []
    for j in range(10): # to predict next 10 req for every batch
        temp.append(req[batch_size*i + j])
    target_req.append(temp)    
target_req = np.array(target_req)

print(train_req.shape,target_req.shape)

(10, 20000) (10, 10)


# Encoder-Decoder

In [109]:
class Encoder(nn.Module):
    def __init__(self, input_dim , hidden_dim , n_layers = 2, dropout = 0):
        super().__init__()
        
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        self.rnn = nn.LSTM(input_dim, hidden_dimension, n_layers, dropout = dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self,train_seq):
        outputs, (hidden, cell) = self.rnn(train_seq)
        return hidden, cell

In [110]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hidden_dim, n_layers, dropout):      
        super().__init__()
        self.output_dim = output_dim
        self.hiden_dim = hidden_dim
        self.n_layers = n_layers
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        
        